In [ ]:
# Standard libraries
import numpy as np
import time
import os

# Computer Vision and Image Processing libraries
import cv2
import torch
import kornia as K

# Specific modules for feature extraction and visualization
import kornia.feature as KF
from kornia_moons.feature import *
from kornia_moons.viz import *

# Utility and progress bar
import tqdm

# Visualization libraries
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
# Process the file
with open("../image_list_hpatches_sequences.txt", 'r') as f:
    lines = [os.path.join("..", line.strip()) for line in f.readlines()]
    lines = sorted(lines)

In [ ]:
num_features = 2048
disk = KF.DISK.from_pretrained("depth").to(device)
output_extension = ".disk"

durations = []

for path in lines:

    image = K.io.load_image(path, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    image2 = K.io.load_image(path, K.io.ImageLoadType.RGB32, device=device)[None, ...]

    with torch.inference_mode():
        inp = torch.cat([image, image2], dim=0)

        starttime = time.time()
        features, _ = disk(inp, num_features, pad_if_not_divisible=True)
        total = round((time.time() - starttime), 2)

        durations.append(total)
        kps1, descs1 = features.keypoints, features.descriptors
        

    with open(path + output_extension, 'wb') as output_file:
        np.savez(
            output_file,
            keypoints=kps1.cpu().numpy(),
            descriptors=descs1.cpu().numpy(),
            )
        
    print(f"Processed {path} in {total} seconds")
